In [2]:
import pandas as pd
import torch
from matplotlib import pyplot as plt
import numpy as np
import time

In [40]:
df = pd.read_csv("ContainsDatasetDrop529.csv")

df

,Unnamed: 0,Date,ContestNumber,Word,ReportedResults,NumberInHardMode,1Try,2Tries,3Tries,4Tries,...,containsq,containsr,containss,containst,containsu,containsv,containsw,containsx,containsy,containsz
0,1,2022-12-31 00:00:00,560,manly,20380,1899,0,2,17,37,...,False,False,False,False,False,False,False,False,True,False
1,2,2022-12-30 00:00:00,559,molar,21204,1973,0,4,21,38,...,False,True,False,False,False,False,False,False,False,False
2,3,2022-12-29 00:00:00,558,havoc,20001,1919,0,2,16,38,...,False,False,False,False,False,True,False,False,False,False
3,4,2022-12-28 00:00:00,557,impel,20160,1937,0,3,21,40,...,False,False,False,False,False,False,False,False,False,False
4,5,2022-12-27 00:00:00,556,condo,20879,2012,0,2,17,35,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,355,2022-01-11 00:00:00,206,drink,153880,3017,1,9,35,34,...,False,True,False,False,False,False,False,False,False,False
354,356,2022-01-10 00:00:00,205,query,107134,2242,1,4,16,30,...,True,True,False,False,True,False,False,False,True,False
355,357,2022-01-09 00:00:00,204,gorge,91477,1913,1,3,13,27,...,False,True,False,False,False,False,False,False,False,False
356,358,2022-01-08 00:00:00,203,crank,101503,1763,1,5,23,31,...,False,True,False,False,False,False,False,False,False,False


In [66]:
from torch.utils.data import random_split
trainY = np.array(df['ReportedResults'])
trainX = np.array(df['ContestNumber'] - 201)

trainY = torch.from_numpy(trainY)
trainX = torch.from_numpy(trainX)

trainY = torch.reshape(trainY, (torch.numel(trainY), 1))
trainY = trainY.double()

trainX = torch.reshape(trainX, (torch.numel(trainX), 1))
trainX = trainX.double()

# trainX, testX = random_split(x, [286, 72])
# trainY, testY = random_split(y, [286, 72])
trainX


tensor([[359.],
        [358.],
        [357.],
        [356.],
        [355.],
        [354.],
        [353.],
        [352.],
        [351.],
        [350.],
        [349.],
        [348.],
        [347.],
        [346.],
        [345.],
        [344.],
        [343.],
        [342.],
        [341.],
        [340.],
        [339.],
        [338.],
        [337.],
        [336.],
        [335.],
        [334.],
        [333.],
        [332.],
        [331.],
        [330.],
        [329.],
        [327.],
        [326.],
        [325.],
        [324.],
        [323.],
        [322.],
        [321.],
        [320.],
        [319.],
        [318.],
        [317.],
        [316.],
        [315.],
        [314.],
        [313.],
        [312.],
        [311.],
        [310.],
        [309.],
        [308.],
        [307.],
        [306.],
        [305.],
        [304.],
        [303.],
        [302.],
        [301.],
        [300.],
        [299.],
        [298.],
        [297.],
        

In [68]:
TensorDS = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(TensorDS, shuffle=True, batch_size=5)
linearModel = torch.nn.Linear(out_features=trainY.size()[1],
in_features=trainX.size()[1], dtype=torch.float64)
optimizer = torch.optim.SGD([linearModel.weight, linearModel.bias],
lr=0.0000001)
loss_fn = torch.nn.MSELoss()

# use a learning rate scheduler to slow down the learning rate when we get close to the optimal solution
# this multiplies the learning rate by 0.99 after each epoch
# adapted from https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99)

start_time = time.time()

for epoch in range(200):
    for batchx, batchy in dl:
        optimizer.zero_grad()
        # Compute the predicted number of deaths for this batch
        predictedy = linearModel(batchx)

        # compute the loss for this batch
        loss = loss_fn(batchy, predictedy)

        # change model params based on loss
        loss.backward()
        optimizer.step()
    scheduler.step()
    # print(epoch, loss_fn(trainY, linearModel(trainX)))

end_time = time.time()
training_time = end_time - start_time

# predictedY = testX @ linearModel.weight.T
# final_r2 = r2_score(testY, predictedY)
# print("r2 score for this model: ", final_r2)
print("Training time: ", training_time, " seconds")

tensor([[226.],
        [337.],
        [ 15.],
        [154.],
        [  9.]], dtype=torch.float64)
tensor([[227.],
        [239.],
        [147.],
        [ 41.],
        [122.]], dtype=torch.float64)
tensor([[103.],
        [ 89.],
        [277.],
        [222.],
        [275.]], dtype=torch.float64)
tensor([[249.],
        [ 71.],
        [ 35.],
        [209.],
        [219.]], dtype=torch.float64)
tensor([[143.],
        [151.],
        [315.],
        [318.],
        [272.]], dtype=torch.float64)
tensor([[140.],
        [261.],
        [189.],
        [166.],
        [ 78.]], dtype=torch.float64)
tensor([[ 29.],
        [ 87.],
        [136.],
        [325.],
        [196.]], dtype=torch.float64)
tensor([[ 56.],
        [ 28.],
        [101.],
        [285.],
        [213.]], dtype=torch.float64)
tensor([[260.],
        [ 65.],
        [306.],
        [323.],
        [296.]], dtype=torch.float64)
tensor([[259.],
        [ 85.],
        [ 68.],
        [114.],
        [142.]], d